In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("fakenewsclassifier.csv")

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data= data.dropna()

In [ ]:
## Get the Independent Features

X=data.loc[:,["title","author","text"]]

In [ ]:
## Get the Dependent features
y=data['label']

In [ ]:
X.shape

(18285, 3)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[608, 1266, 424, 1635, 4402, 4086, 4319, 3023, 4132, 652],
 [2271, 703, 1354, 2625, 3085, 3674, 4189],
 [351, 3266, 2715, 4374],
 [1853, 1290, 3530, 3611, 690, 4082],
 [4023, 3085, 1945, 3870, 4749, 4306, 3085, 3170, 3423, 2290],
 [4351,
  2396,
  259,
  4730,
  2571,
  4487,
  2363,
  4262,
  1321,
  2331,
  2906,
  1580,
  4715,
  842,
  4189],
 [1198, 2461, 4960, 1920, 4322, 404, 4209, 3390, 179, 3351, 4479],
 [4004, 1865, 4483, 2144, 1962, 2572, 4487, 798, 179, 3351, 4479],
 [381, 328, 3185, 4039, 197, 1318, 4971, 3832, 4487, 930],
 [986, 2948, 3892, 4200, 4419, 2035, 3921, 2186],
 [3784, 2003, 1965, 1042, 3147, 4090, 4599, 3175, 1737, 2186, 4294],
 [3611, 3200, 4402, 1318, 4487, 1962],
 [2037, 4944, 4475, 3316, 2929, 2088, 3336, 3746, 1987],
 [3885, 178, 556, 2675, 2690, 3517, 1793, 179, 3351, 4479],
 [557, 445, 4778, 1263, 4249, 179, 3351, 4479],
 [1912, 3491, 4495, 59, 103, 4663, 2371, 1530, 774, 4675],
 [2916, 2018, 703],
 [2822, 3613, 2015, 1133, 4487, 4040, 732, 4189],
 [218

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3023 4132  652]
 [   0    0    0 ... 3085 3674 4189]
 [   0    0    0 ... 3266 2715 4374]
 ...
 [   0    0    0 ...  179 3351 4479]
 [   0    0    0 ... 4075 1845 3858]
 [   0    0    0 ...  280 3931 2200]]


In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.4689 - accuracy: 0.7858 - val_loss: 0.1954 - val_accuracy: 0.9198
Epoch 2/10
192/192 [==============================] - 6s 33ms/step - loss: 0.1388 - accuracy: 0.9454 - val_loss: 0.2040 - val_accuracy: 0.9241
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0913 - accuracy: 0.9654 - val_loss: 0.2140 - val_accuracy: 0.9180
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0673 - accuracy: 0.9732 - val_loss: 0.2907 - val_accuracy: 0.9110
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0464 - accuracy: 0.9854 - val_loss: 0.3091 - val_accuracy: 0.9115
Epoch 6/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0289 - accuracy: 0.9901 - val_loss: 0.3350 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0189 - accuracy: 0.9951 - val_loss: 0.3855 - val_accuracy: 0.9158

In [41]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3080,  339],
       [ 183, 2433]])

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9135045567522784